In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import timeit
from datetime import datetime
from tensorflow import keras
from tensorflow.keras import layers

%load_ext tensorboard

print("Using TensorFlow version %s" % tf.__version__)
import tensorflow.experimental.numpy as tnp

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Using TensorFlow version 2.5.0-dev20210102


In [5]:
#A Sequential model is appropriate for a plain stack of layers where each layer has exactly one input tensor and one output tensor.

# Method 1 
# Define Sequential model with 3 layers
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu", name="layer1"),
        layers.Dense(3, activation="relu", name="layer2"),
        layers.Dense(4, name="layer3"),
    ]
)
# Call model on a test input
x = tf.ones((3, 3))
y = model(x)
print("model.layers ",model.layers)
model.summary()

# Method 2
# Create 3 layers
layer1 = layers.Dense(2, activation="relu", name="layer1")
layer2 = layers.Dense(3, activation="relu", name="layer2")
layer3 = layers.Dense(4, name="layer3")

# Call layers on a test input
x = tf.ones((3, 3))
y = layer3(layer2(layer1(x)))

# Method 3:
model = keras.Sequential()
model.add(layers.Dense(2, activation="relu"))
model.add(layers.Dense(3, activation="relu"))
model.add(layers.Dense(4))

# you can add name argument for tensorboard
model = keras.Sequential(name="my_sequential")
model.add(layers.Dense(2, activation="relu", name="layer1"))
model.add(layers.Dense(3, activation="relu", name="layer2"))
model.add(layers.Dense(4, name="layer3"))

# Method 4:
# Each layer in keras need to know inputs/input shape in order to create weights
# So in all of the above, before you call test input, ;layer1.weights and so is empty
# but once the model is 'built'(has weights), you could call model.summary()

model.layers  [<tensorflow.python.keras.layers.core.Dense object at 0x000001A65F526FA0>, <tensorflow.python.keras.layers.core.Dense object at 0x000001A65F4D3CD0>, <tensorflow.python.keras.layers.core.Dense object at 0x000001A65F52F3A0>]
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (Dense)               (3, 2)                    8         
_________________________________________________________________
layer2 (Dense)               (3, 3)                    9         
_________________________________________________________________
layer3 (Dense)               (3, 4)                    16        
Total params: 33
Trainable params: 33
Non-trainable params: 0
_________________________________________________________________


In [6]:
# However, if you want to see summry in between while making layers, start with input shape, then add, then summary, then add, then summary and so on

model = keras.Sequential()
model.add(keras.Input(shape=(4,)))
model.add(layers.Dense(2, activation="relu"))
#Or just model.add(layers.Dense(2, activation="relu", input_shape=(4,)))

model.summary()



Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 2)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


In [7]:
# Use add+summary to keep checking the model output while making
# functional API == Sequential API which is 'built' but still it can't have multiple inputs/outputs


# you can then do following
#1. Extract output from intermediate layers
initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
    ]
)
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=[layer.output for layer in initial_model.layers],
)

# Call feature extractor on test input.
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)

In [ ]:
# Transfer learning:
# 1. Freeze all layers except last one and update its weight while compiling+fitting ( for layer in model.layers[:-1]: layer.trainable = False)
# 2. Freeze a stack of pre-trained layers and just train of remaining new added layers:
# Load a convolutional base with pre-trained weights
base_model = keras.applications.Xception(
    weights='imagenet',
    include_top=False,
    pooling='avg')

# Freeze the base model
base_model.trainable = False

# Use a Sequential model to add a trainable classifier on top
model = keras.Sequential([
    base_model,
    layers.Dense(1000),
])

# Compile & train
model.compile(...)
model.fit(...)